In [ ]:
# 初始化环境
import os
from openai import OpenAI

os.environ['OPENAI_BASE_URL'] = 'https://key.wenwen-ai.com/v1'
os.environ['OPENAI_API_KEY'] = 'sk-GYYKJ77h2qJ9YfQV46WAHALlqtz1N17Ma3HpyOw1VHKkb0zW'


client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-image",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content":"[全景]镜头，背景是在一个路边内，周边环境是道路旁有路灯，周围较安静。图片参考人物（虚拟角色）牵着一个[青年，女性]（主控角色）的手，表情为满眼爱意。[青年，女性]（主控角色） 穿着蓝色短裙，站在参考图角色前面，背对着镜头微微低头。  生成画风参考入参中的画风，虚拟角色需保持人物形象特征。 超清画质，超高细节，五官清晰，眼神明亮，身体比例正常，画面质量和谐。图片宽高比例【2:3】"}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
    # top_p=1,
    # stop="!",
    # seed=1234567,
    # user="user_1234567",
)

print(completion.model_dump_json())


In [3]:
import json
import time
import requests
from PIL import Image
from io import BytesIO
from collections import Counter
from openai import OpenAI

def get_image_ratio(image_url):
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        width, height = img.size
        ratio = width / height
        return round(ratio, 2)
    except Exception as e:
        print(f'Error processing image: {e}')
        return None

client = OpenAI()
ratios = []
N = 2
for i in range(N):
    try:
        print(f'Processing iteration {i+1}/{N}...')
        completion = client.chat.completions.create(
            model="gpt-4o-image",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "[全景]镜头，背景是在一个路边内，周边环境是道路旁有路灯，周围较安静。图片参考人物（虚拟角色）牵着一个[青年，女性]（主控角色）的手，表情为满眼爱意。[青年，女性]（主控角色） 穿着蓝色短裙，站在参考图角色前面，背对着镜头微微低头。  生成画风参考入参中的画风，虚拟角色需保持人物形象特征。 超清画质，超高细节，五官清晰，眼神明亮，身体比例正常，画面质量和谐。图片宽高比例【2:3】"}
            ],
            n=1,
            max_tokens=2048,
            temperature=0.8,
        )
        response_data = json.loads(completion.model_dump_json())
        image_url = response_data['choices'][0]['message']['content'].split('(')[1].split(')')[0]
        ratio = get_image_ratio(image_url)
        if ratio:
            ratios.append(ratio)
        time.sleep(1)
    except Exception as e:
        print(f'Error in iteration {i+1}: {e}')
        continue

print("\n=== 分析结果 ===")
print(f'总共成功处理图片数量: {len(ratios)}')
ratio_counter = Counter([str(r) for r in ratios])
print("\n长宽比分布:")
for ratio, count in ratio_counter.most_common():
    percentage = (count / len(ratios)) * 100
    print(f'比例 {ratio}: {count}次 ({percentage:.1f}%)')
target_ratio = 2/3
tolerance = 0.05
matches = sum(1 for r in ratios if abs(r - target_ratio) <= tolerance)
match_percentage = (matches / len(ratios)) * 100 if ratios else 0
print(f'\n符合2:3比例(误差范围±5%)的图片数量: {matches}/{len(ratios)} ({match_percentage:.1f}%)')

Processing iteration 1/2...
Processing iteration 2/2...

=== 分析结果 ===
总共成功处理图片数量: 2

长宽比分布:
比例 0.67: 2次 (100.0%)

符合2:3比例(误差范围±5%)的图片数量: 2/2 (100.0%)
